In [1]:
import numpy as np 
import pandas as pd


import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.optimizers import RMSprop

# Load Data

In [2]:
# Import data into a dataframe
train= pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test= pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [3]:
print(f"Train Data Info: {train.info()}\n")
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB
Train Data Info: None



,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
print(f"Test Data Info: {test.info()}\n")
test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 167.5 MB
Test Data Info: None



,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Train dataset contains the target and features columns, it has a shape of 785 columns and 42000 rows/observations,while the test dataset contains just the features columns, and is a shape of 784 columns and 28000 rows/observations.

# Split Training Dataset to Targets and Features

In [5]:
target= train["label"]
features= train.drop("label", axis=1)

In [6]:
print(f"Target type: {type(target)}\n")
print(f"Features type: {type(features)}")

Target type: <class 'pandas.core.series.Series'>

Features type: <class 'pandas.core.frame.DataFrame'>


In [7]:
print(target.head())
features.head()

0    1
1    0
2    1
3    4
4    0
Name: label, dtype: int64


,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Preprocessing the Datasets
- Data Noramlization
- Scaling Data

In [8]:
def preprocess_data(data):
    # Normalize the data
    feat_norm = data / 255.0

    # Initialize a new StandardScaler instance
    scaler = StandardScaler()

    # Fit the scaler to the data
    scaler.fit(feat_norm)

    # Transform the data
    feat_scaled = scaler.transform(feat_norm)
    
    return feat_scaled


In [9]:
feat_train= preprocess_data(features)
feat_test= preprocess_data(test)

In [10]:
# Datatype
print(f"Train Scaled DataType: {type(feat_train)}")
print(f"Test Scaled DataType: {type(feat_test)}")

Train Scaled DataType: <class 'numpy.ndarray'>
Test Scaled DataType: <class 'numpy.ndarray'>


In [11]:
# Printing the Data
print(f"Train Scaled Data: \n {feat_train[0:5]}\n")
print(f"Test Scaled Data: \n {feat_test[0:5]}")

Train Scaled Data: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

Test Scaled Data: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [12]:
print(f"Train Scaled Data Shape: \n {feat_train.shape}\n")
print(f"Test Scaled Data Shape: \n {feat_test.shape}")

Train Scaled Data Shape: 
 (42000, 784)

Test Scaled Data Shape: 
 (28000, 784)
